In [1]:
import csv

input_file = 'formated_data.txt'
output_file = 'output.csv'

with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8', newline='') as f_out:
    lines = f_in.readlines()
    header = lines[0].strip().split('\t')

    writer = csv.writer(f_out)
    writer.writerow(header)

    for line in lines[1:]:
        row = line.strip().split('\t')
        writer.writerow(row)

print(f'Данные сохранены в {output_file}')


FileNotFoundError: [Errno 2] No such file or directory: 'formated_data.txt'

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('output.csv')

def find_significant_fields(df, var_threshold=1e-3, missing_threshold=0.1, unique_threshold=1):
    significant_fields = []
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            missing_ratio = df[col].isna().mean()
            if missing_ratio > missing_threshold:
                continue

            unique_vals = df[col].nunique(dropna=True)
            if unique_vals <= unique_threshold:
                continue

            variance = df[col].var()
            if pd.isna(variance) or variance < var_threshold:
                continue

            significant_fields.append(col)
    return significant_fields

significant_cols = find_significant_fields(df)

df_significant = df[significant_cols]

output_filename = 'cleaned_dataset.csv'
df_significant.to_csv(output_filename, index=False, encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: 'output.csv'

In [ ]:
df_cleaned = df_significant.drop_duplicates()
df_cleaned = df_cleaned.dropna()

def remove_outliers(df, n_std=3):
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        lower_bound = mean - n_std * std
        upper_bound = mean + n_std * std
        df = df.loc[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

df_cleaned = remove_outliers(df_cleaned, n_std=3)

print(f"Размер датасета после очистки: {df_cleaned.shape}")

df_cleaned.to_csv('cleaned_dataset_no_outliers.csv', index=False)
print("Очищенный от выбросов датасет сохранён в cleaned_dataset_no_outliers.csv")


In [ ]:
import pandas as pd
import numpy as np

corr_matrix = df_cleaned.corr().abs()

cols_to_drop = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        col1 = corr_matrix.columns[i]
        col2 = corr_matrix.columns[j]
        if corr_matrix.iloc[i, j] > 0.98:
            cols_to_drop.add(col2)

print(f"Столбцы, которые будут удалены из-за полной корреляции: {cols_to_drop}")

df_cleaned = df_significant.drop(columns=list(cols_to_drop))

print(f"Размер датафрейма до удаления: {df_significant.shape}")
print(f"Размер датафрейма после удаления: {df_cleaned.shape}")

df_cleaned.to_csv('cleaned_dataset_no_duplicates.csv', index=False)
print("Очищенный датасет сохранён в cleaned_dataset_no_duplicates.csv")